In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import models
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchinfo import summary
from torch.cuda.amp import GradScaler

from albumentations.pytorch import ToTensorV2
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Definizione del Dataset e Caricamento dei Dati

In questa sezione, prepariamo il nostro dataset per l'addestramento e la valutazione del modello. Le operazioni principali sono:

1.  **Configurazione Iniziale**: Vengono definiti i percorsi (`root_color_train`, `root_depth_test`, ecc.) alle directory contenenti le immagini a colori e le mappe di profondità. Vengono inoltre impostati gli iperparametri fondamentali come la dimensione delle immagini (`img_size`), la grandezza dei batch (`batch_size`) e i parametri per l'ottimizzatore (`lr`, `weight_decay`).

2.  **Creazione della Classe `AntiSpoofDataset`**: Poiché il nostro modello utilizza un input multimodale (**immagini RGB** e **mappe di profondità**), creiamo una classe `Dataset` personalizzata. Questa classe gestisce:
    *   Il caricamento simultaneo di un'immagine a colori e della sua corrispondente mappa di profondità.
    *   Il ridimensionamento di entrambe le immagini a una dimensione uniforme (`img_size`).
    *   L'applicazione di trasformazioni e data augmentation (definite in una cella successiva).
    *   La conversione delle immagini in tensori PyTorch pronti per essere inviati al modello.

3.  **Funzione `load_paths` e Caricamento**: La funzione `load_paths` esamina le directory specificate, identifica i campioni come `real` (etichetta 0) o `fake` (etichetta 1) in base al nome del file e crea le liste di percorsi e etichette. Infine, questa funzione viene eseguita per caricare i dati di training e test, stampando un riepilogo del numero di campioni per ogni set e per ogni classe.

In [7]:
root_color_train = "/content/drive/MyDrive/dataset_casia/train_img/color"
root_depth_train = "/content/drive/MyDrive/dataset_casia/train_img/depth_midas_train"

root_color_test = "/content/drive/MyDrive/dataset_casia/test_img/color"
root_depth_test = "/content/drive/MyDrive/dataset_casia/test_img/depth_midas_test"

img_size = 224
batch_size = 24
lr = 1e-4
weight_decay = 5e-4
patience = 15
use_mixed_precision = True

In [ ]:
class AntiSpoofDataset(Dataset):
    def __init__(self, color_paths, depth_paths, labels, transform=None):
        self.color_paths = color_paths
        self.depth_paths = depth_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.color_paths)

    def __getitem__(self, idx):
        # Carica immagini
        img = cv2.imread(self.color_paths[idx]) #(H,W,3)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #(H,W,3)
        depth = cv2.imread(self.depth_paths[idx], cv2.IMREAD_GRAYSCALE) #(H,W)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        # Resize
        img = cv2.resize(img, (img_size, img_size))
        depth = cv2.resize(depth, (img_size, img_size))

        if self.transform:
            # Augmentation
            #augmented è un dizionario in cui le immagini rgb e le corrispettive mappe (chiavi del dizionario) sono sottoposte alle stesse trasformazioni geometriche
            augmented = self.transform(image=img, mask=depth) 
            img = augmented["image"]    # Tensor [3, H, W] 
            depth = augmented["mask"]   # Numpy [H, W] (valori uint8 [0-255])

            # Se albumentations non converte da numpy a tensore, converto numpy → Tensor e normalizzo
            if isinstance(depth, np.ndarray):
                depth = torch.from_numpy(depth).float() / 255.0
            #Se albumentations converte da numpy a tensore, normalizzo
            else:
                # Se albumentations già converte (versioni diverse)
                depth = depth.float() / 255.0

            depth = depth.unsqueeze(0)  # [1, H, W]
        else:
            # Senza augmentation
            img = torch.from_numpy(img.transpose(2, 0, 1)).float() / 255.0 #(H,W,3) --> (3,H,W)
            depth = torch.from_numpy(depth).float() / 255.0
            depth = depth.unsqueeze(0)

        return img, depth, label


def load_paths(root_color, root_depth):
    color_paths, depth_paths, labels = [], [], []

    for f in os.listdir(root_color):
        if not f.lower().endswith((".jpg", ".png")):
            continue
        f_lower = f.lower()

        if "real" in f_lower:
            label = 0
        elif "fake" in f_lower:
            label = 1
        else:
            continue

        color_path = os.path.join(root_color, f)
        depth_path = os.path.join(root_depth, f)

        if os.path.exists(depth_path):
            color_paths.append(color_path)
            depth_paths.append(depth_path)
            labels.append(label)

    print(f"Caricati {len(color_paths)} campioni da {root_color}")
    print(f"Real: {labels.count(0)} | Fake: {labels.count(1)}")
    return color_paths, depth_paths, labels


train_color, train_depth, train_labels = load_paths(root_color_train, root_depth_train)
test_color, test_depth, test_labels = load_paths(root_color_test, root_depth_test)

print(f"\nTrain: {len(train_color)} | Test: {len(test_color)}")

Caricati 2408 campioni da /content/drive/MyDrive/dataset_casia/train_img/color
Real: 591 | Fake: 1817
Caricati 1655 campioni da /content/drive/MyDrive/dataset_casia/test_img/color
Real: 404 | Fake: 1251

Train: 2408 | Test: 1655


### Data Augmentation e Creazione dei DataLoader

Questa cella si occupa di due compiti fondamentali: definire le strategie di data augmentation e creare i `DataLoader` di PyTorch, che preparano i dati in lotti (batch) per l'addestramento e la valutazione.

#### Data Augmentation con Albumentations
Per rendere il modello più robusto e capace di generalizzare a condizioni del mondo reale, viene applicata una pipeline di **data augmentation** molto ricca al solo set di addestramento, utilizzando la libreria `Albumentations`. Le trasformazioni sono state scelte per simulare scenari di attacco comuni:

-   **Variazioni di Illuminazione**: `RandomBrightnessContrast`, `RandomGamma` e `HueSaturationValue` per simulare diverse condizioni di luce ambientale.
-   **Artefatti di Display/Stampa**: `GaussNoise`, `ISONoise`, `MotionBlur` e `GaussianBlur` per imitare il rumore e le sfocature introdotte da schermi, proiettori o stampe di bassa qualità.
-   **Diverse Angolazioni e Prospettive**: `ShiftScaleRotate` e `Perspective` per simulare riprese del volto da diverse angolazioni.
-   **Qualità del Sensore**: `CLAHE` e `Sharpen` per simulare l'effetto di diverse fotocamere.

L'argomento `additional_targets={'mask': 'mask'}` è cruciale: garantisce che le trasformazioni geometriche (come rotazioni o ritagli) vengano applicate in modo identico sia all'immagine RGB sia alla mappa di profondità, mantenendole perfettamente allineate.

Il set di test, invece, subisce solo la normalizzazione e la conversione in tensore per garantire una valutazione oggettiva e riproducibile delle performance del modello.

#### Gestione dello Sbilanciamento delle Classi
Come osservato in precedenza, il dataset è sbilanciato, con un numero maggiore di campioni "fake". Per evitare che il modello diventi "pigro" e prediliga la classe maggioritaria, viene utilizzato un `WeightedRandomSampler`. Questo campionatore assegna un peso maggiore ai campioni della classe meno rappresentata (in questo caso, "real"), assicurando che durante l'addestramento il modello veda un numero più bilanciato di esempi per ogni classe.

#### Creazione dei DataLoader
Infine, vengono creati i `DataLoader` per i set di training e test. Questi oggetti gestiscono il caricamento efficiente dei dati in batch, sfruttando il campionatore pesato per il training e mantenendo l'ordine sequenziale per il testing (`shuffle=False`). L'opzione `pin_memory=True` viene usata per velocizzare il trasferimento dei dati sulla GPU.

In [9]:
train_tf = A.Compose([
    # Simula diverse condizioni di illuminazione
    A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.7),
    A.RandomGamma(gamma_limit=(70, 130), p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.6),

    # Simula artefatti di stampa/display
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.4),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.3),
    A.MotionBlur(blur_limit=7, p=0.4),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),

    # Simula diverse angolazioni
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=0.6),
    A.Perspective(scale=(0.05, 0.1), p=0.4),

    # Simula diversi dispositivi di cattura
    A.CLAHE(clip_limit=2.0, p=0.3),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.3),

    # Augmentazioni base
    A.HorizontalFlip(p=0.5),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),

    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'mask': 'mask'})


test_tf = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'mask': 'mask'})

train_dataset = AntiSpoofDataset(train_color,train_depth,train_labels,train_tf)
test_dataset = AntiSpoofDataset(test_color,test_depth,test_labels,test_tf)

class_counts = [train_labels.count(0), train_labels.count(1)]
class_weights = [1.0 / c for c in class_counts]
sample_weights = [class_weights[label] for label in train_labels]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=2,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2,pin_memory=True)


### Architettura del Modello Anti-Spoofing

Questa sezione definisce l'architettura completa del nostro modello di classificazione. Il design si basa su un approccio **multimodale** che sfrutta sia le informazioni cromatiche (RGB) sia quelle di profondità per massimizzare l'accuratezza. L'architettura è composta da diversi moduli chiave:

#### `RGBEncoder`
Questo modulo è responsabile dell'elaborazione delle immagini a colori. La sua architettura è così strutturata:
-   **Backbone**: Utilizza le feature convoluzionali di **EfficientNet-B0**, pre-addestrato su ImageNet. Questa scelta fornisce un potente estrattore di feature senza un costo computazionale eccessivo.
-   **`ChannelAttention`**: Dopo l'estrazione delle feature, viene applicato un meccanismo di attenzione sui canali. Questo modulo, ispirato a CBAM (Convolutional Block Attention Module), impara a pesare dinamicamente i canali delle feature map, permettendo al modello di concentrarsi sulle informazioni più rilevanti e sopprimere quelle meno utili.
-   **Head di Proiezione**: Le feature raffinate vengono poi processate da un blocco di classificazione (head) composto da strati lineari, normalizzazione e dropout, che le proietta in uno spazio latente a 128 dimensioni.

#### `DepthEncoder`
Questo modulo, progettato su misura, elabora le mappe di profondità in scala di grigi. La sua architettura è pensata per catturare informazioni a diverse scale:
-   **Feature Extractor Multi-Scala**: Consiste in una serie di blocchi convoluzionali che estraggono feature a risoluzioni decrescenti (`f1`, `f2`, `f3`).
-   **Fusione Multi-Scala**: Le feature map estratte a diverse profondità vengono ridimensionate alla stessa risoluzione spaziale e concatenate. Questo permette al modello di combinare dettagli fini (da feature map a risoluzione più alta) con informazioni contestuali più ampie (da feature map a risoluzione più bassa).
-   **Head di Proiezione**: Similmente all'encoder RGB, un blocco finale proietta le feature di profondità fuse in uno spazio latente a 128 dimensioni.

#### `CrossModalAttention`
Questo è il cuore dell'interazione tra le due modalità. Vengono utilizzati due moduli di attenzione incrociata:
1.  **`rgb_to_depth`**: Usa le feature RGB come *query* e le feature di profondità come *key* e *value*. In pratica, "chiede" alle feature di profondità quali siano le più rilevanti per contestualizzare le informazioni RGB.
2.  **`depth_to_rgb`**: Fa l'opposto, usando la profondità come *query* per pesare le feature RGB.

Questo meccanismo permette a ciascuna modalità di arricchirsi con le informazioni complementari fornite dall'altra.

#### `AntiSpoofingModel` (Modello Principale)
Il modello finale assembla i componenti precedenti:
1.  **Estrazione Indipendente**: Le immagini RGB e di profondità vengono processate dai rispettivi encoder per ottenere i vettori di feature iniziali (`rgb_feat`, `depth_feat`).
2.  **Arricchimento Incrociato**: Vengono calcolate le feature "attese" (`rgb_attended`, `depth_attended`) tramite i moduli di `CrossModalAttention`.
3.  **Concatenazione e Fusione**: I quattro vettori di feature (originali e attesi) vengono concatenati in un unico vettore di dimensione 512.
4.  **Classificazione Finale**: Un blocco di fusione, composto da strati densi con dropout e normalizzazione, processa il vettore combinato per produrre le feature finali a 128 dimensioni. Infine, un classificatore lineare produce i *logit* per le due classi ("real" e "fake").

Il modello può restituire opzionalmente anche le feature finali, utili per funzioni di loss più complesse come la loss contrastiva.

In [10]:
class ChannelAttention(nn.Module):
  def __init__(self,channels,reduction=16):
    super().__init__()

    self.avg_pool = nn.AdaptiveAvgPool2d(1) # pool medio globale
    self.max_pool = nn.AdaptiveMaxPool2d(1) # pool max globale

    #Bottleneck
    self.fc = nn.Sequential(
        nn.Linear(channels,channels//reduction,bias=False),
        nn.ReLU(inplace=True),
        nn.Linear(channels//reduction,channels,bias=False)
    )
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    b, c, _, _ = x.size()
    avg_out = self.avg_pool(x).view(b, c)  # [B, 1280, 7, 7] → [B, 1280, 1, 1] → [B, 1280]
    max_out = self.max_pool(x).view(b, c)  # [B, 1280, 7, 7] → [B, 1280, 1, 1] → [B, 1280]
    avg_out = self.fc(avg_out)  # [B, 1280] → [B, 80] → [B, 1280]
    max_out = self.fc(max_out)  # [B, 1280] → [B, 80] → [B, 1280]
    out = self.sigmoid(avg_out + max_out)  # [B, 1280] → somma → sigmoid
    out = out.view(b, c, 1, 1)              # [B, 1280, 1, 1]
    return x * out  # [B, 1280, 7, 7] * [B, 1280, 1, 1] → [B, 1280, 7, 7]


class RGBEncoder(nn.Module):
  def __init__(self,output_dim=128):
    super().__init__()

    base = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

    #backbone dell'encoder RGB --> EfficentNet-B0 net addestrato su ImageNet ([B,3,224,224] --> [B,1280,7,7])
    self.feature_extractor = base.features
    #([B,1280,7,7] --> [B,1280,7,7])
    self.attention = ChannelAttention(1280)
    # ([B,1280],[B120])
    self.head = nn.Sequential(
        nn.Linear(1280, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(256, 128)
    )

    self.output_dim = output_dim

  def forward(self,x):
    x = self.feature_extractor(x)
    x = self.attention(x)
    x = F.adaptive_avg_pool2d(x,1).flatten(1) #([B,1280,7,7] --> [B,1280])
    return self.head(x)

class DepthEncoder(nn.Module):
  def __init__(self,output_dim=128):
    super().__init__()

    # [B,1,224,224] → [B,32,112,112]
    self.depth1 = nn.Sequential(
        nn.Conv2d(1,32,3,2,1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32,32,3,1,1),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )

    # [B,32,112,112] --> [B,64,56,56]
    self.depth2 = nn.Sequential(
        nn.Conv2d(32,64,3,2,1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64,64,3,1,1),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )

    # [B,64,56,56] --> [B,128,28,28]
    self.depth3 = nn.Sequential(
        nn.Conv2d(64,128,3,2,1),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )

    self.fusion = nn.Sequential(
        nn.Conv2d(224,128,1),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )

    self.upsample1 = nn.Upsample(scale_factor=4,mode='bilinear',align_corners=False)
    self.upsample2 = nn.Upsample(scale_factor=2,mode='bilinear',align_corners=False)

    self.global_pool = nn.AdaptiveAvgPool2d(1)

    self.fc = nn.Sequential(
        nn.Linear(128,128)
    )

    self.output_dim = output_dim

  def forward(self,x):
    f1 = self.depth1(x)   # [B,1,224,224] → [B,32,112,112]
    f2 = self.depth2(f1)  # Usa f1 (32 canali)
    f3 = self.depth3(f2)  # Usa f2 (64 canali)

    f1_up = self.upsample1(f1) # [B,32,112,112] --> [B,32,448,448]
    f2_up = self.upsample2(f2) # [B,64,56,56] --> [B,64,112,112]

    target_size = f3.shape[2:] # (28,28)
    f1_up = F.interpolate(f1_up,size=target_size,mode='bilinear',align_corners=False) # [B,32,448,448] --> [b,32,64,64]
    f2_up = F.interpolate(f2_up,size=target_size,mode='bilinear',align_corners=False) # [B,64,112,112] --> [B,64,28,28]

    #concatenazione multi-scala
    multi_scale = torch.cat([f1_up,f2_up,f3],dim=1) # [B,32,28,28] + [B,64,28,28] + [B,128,28,28] = [B,224,28,28]

    fused = self.fusion(multi_scale) # [B,224,28,28] --> [B,128,28,28]

    x = F.adaptive_avg_pool2d(fused, 1).flatten(1)  # [B,128,28,28] → [B,128]
    return self.fc(x)  # [B,128]


class CrossModalAttention(nn.Module):
  def __init__(self,dim):
    super().__init__()

    self.query = nn.Linear(dim,dim)
    self.key = nn.Linear(dim,dim)
    self.value = nn.Linear(dim,dim)
    self.scale = dim ** -0.5
    self.out_proj = nn.Linear(dim, dim)

  def forward(self,rgb_x,depth_x):
    q = self.query(rgb_x).unsqueeze(1) # [B, 128] → [B, 1, 128]
    k = self.key(depth_x).unsqueeze(1) # [B, 128] → [B, 1, 128]
    v = self.value(depth_x).unsqueeze(1) # [B, 128] → [B, 1, 128]

    attn = (q @ k.transpose(-2,-1)) * self.scale
    attn = F.softmax(attn,dim=-1) # normalizzo in [0,1]
    out = (attn @ v).squeeze(1)  # [B, 1, 1] @ [B, 1, 128] → [B, 128]
    return self.out_proj(out)


class AntiSpoofingModel(nn.Module):
  def __init__(self,rgb_encoder,depth_encoder):
    super().__init__()
    self.rgb_encoder = rgb_encoder
    self.depth_encoder = depth_encoder

    self.rgb_to_depth = CrossModalAttention(rgb_encoder.output_dim)
    self.depth_to_rgb = CrossModalAttention(depth_encoder.output_dim)

    fusion_dim = rgb_encoder.output_dim * 2 + depth_encoder.output_dim * 2
    self.fusion = nn.Sequential(
        nn.Linear(fusion_dim, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.3),
    )

    self.classifier = nn.Linear(128,2)

  def forward(self,rgb_x,depth_x,return_features=False):
    rgb_feat = self.rgb_encoder(rgb_x)
    depth_feat = self.depth_encoder(depth_x)

    rgb_attended = self.rgb_to_depth(rgb_feat,depth_feat)
    depth_attended = self.depth_to_rgb(depth_feat,rgb_feat)

    x = torch.cat(
        [
            rgb_feat,rgb_attended,
            depth_feat,depth_attended
        ],
        dim=1
    )

    features = self.fusion(x)
    logits = self.classifier(features)

    if return_features:
            return logits, features
    return logits

In [11]:
model = AntiSpoofingModel(
    RGBEncoder(),
    DepthEncoder()
).to(device)

summary(
    model,
    input_data=[
        torch.randn(1, 3, 224, 224).to(device),   # RGB input
        torch.randn(1, 1, 224, 224).to(device)    # Depth input
    ],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    depth=3,
    verbose=0
)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 239MB/s]


Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Trainable
AntiSpoofingModel                                            [1, 3, 224, 224]          [1, 2]                    --                        True
├─RGBEncoder: 1-1                                            [1, 3, 224, 224]          [1, 128]                  --                        True
│    └─Sequential: 2-1                                       [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        True
│    │    └─Conv2dNormActivation: 3-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         928                       True
│    │    └─Sequential: 3-2                                  [1, 32, 112, 112]         [1, 16, 112, 112]         1,448                     True
│    │    └─Sequential: 3-3                                  [1, 16, 112, 112]         [1, 24, 56, 56]           16,714            

### Funzione di Loss Ibrida: `ContrastativeFocalLoss`

Per addestrare efficacemente il nostro modello, non ci affidiamo a una semplice Cross-Entropy, ma implementiamo una **funzione di loss ibrida** personalizzata che combina due potenti concetti: la **Focal Loss** e la **Contrastive Loss**. Questa scelta è motivata dalla necessità di affrontare due sfide comuni: lo sbilanciamento delle classi e la difficoltà nel distinguere esempi "difficili".

#### Focal Loss
La componente `focal_loss` affronta il problema dello sbilanciamento delle classi e la presenza di esempi "facili" e "difficili". A differenza della Cross-Entropy standard, la Focal Loss riduce dinamicamente il peso degli esempi che il modello classifica già correttamente (quelli "facili"). Questo permette all'ottimizzatore di **concentrarsi sugli esempi più difficili e informativi**, che sono spesso quelli al confine tra le classi "real" e "fake". I parametri chiave sono:
-   `alpha`: Pesa l'importanza delle classi positive/negative.
-   `gamma`: Modula la focalizzazione; un `gamma` più alto aumenta l'effetto di down-weighting sugli esempi facili.

#### Contrastive Loss (Supervised)
La componente `contrastive_loss` opera non sui *logit* finali, ma direttamente sullo **spazio delle feature** a 128 dimensioni prodotte dal modello. L'obiettivo è quello di strutturare questo spazio latente in modo che sia semanticamente significativo. In particolare, questa loss:
-   **Avvicina** le feature di campioni appartenenti alla stessa classe (es. due volti "real" diversi).
-   **Allontana** le feature di campioni appartenenti a classi diverse (es. un volto "real" e uno "fake").

Questo approccio, noto come *Supervised Contrastive Learning*, aiuta il modello a imparare rappresentazioni più robuste e discriminative, migliorando la sua capacità di generalizzazione. Il parametro `temperature` controlla la separazione tra le classi nello spazio latente.

#### Combinazione
La loss finale è una somma pesata delle due componenti:
`Total Loss = focal_loss + contrast_weight * contrastive_loss`

Durante l'**addestramento**, vengono utilizzate entrambe le componenti per ottimizzare sia la classificazione sia la struttura dello spazio delle feature. Durante la **valutazione**, viene usata solo la `focal_loss` per calcolare una metrica di performance coerente, poiché la componente contrastiva dipende dalla composizione del batch.

In [12]:
class ContrastativeFocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, contrast_weight=0.3, temperature=0.5):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.contrast_weight = contrast_weight
        self.temperature = temperature

    def focal_loss(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

    def contrastive_loss(self, features, labels):
        # Normalizza features per calcolare similarità
        features = F.normalize(features, dim=1)

        # Matrice di similarità (cosine similarity)
        sim_matrix = torch.matmul(features, features.T) / self.temperature

        # Maschera per positive pairs (stessa classe)
        labels = labels.view(-1, 1)
        mask = torch.eq(labels, labels.T).float().to(features.device)

        # Rimuovi diagonale (non confrontare sample con se stesso)
        mask = mask - torch.eye(mask.size(0)).to(features.device)

        # Contrastive loss: avvicina same-class, allontana different-class
        exp_sim = torch.exp(sim_matrix)
        log_prob = sim_matrix - torch.log(exp_sim.sum(1, keepdim=True) + 1e-8)

        # Media solo sui positive pairs
        mean_log_prob_pos = (mask * log_prob).sum(1) / (mask.sum(1) + 1e-8)
        loss = -mean_log_prob_pos.mean()
        return loss

    def forward(self, logits, features=None, targets=None):
        # Caso: criterion(out, y)
        if targets is None and features is not None and features.dtype == torch.long:
            targets = features
            features = None

        # Solo focal loss (test)
        if features is None:
            return self.focal_loss(logits, targets)

        # Focal + contrastive (train)
        focal = self.focal_loss(logits, targets)
        contrastive = self.contrastive_loss(features, targets)
        return focal + self.contrast_weight * contrastive

### Ottimizzatore, Scheduler e Early Stopping

In questa sezione, configuriamo tutti gli strumenti necessari per il processo di addestramento: l'ottimizzatore che aggiorna i pesi del modello, lo scheduler che gestisce il learning rate e un meccanismo di early stopping per prevenire l'overfitting e salvare il modello migliore.

#### Ottimizzatore e Funzione di Loss
-   **Ottimizzatore `AdamW`**: È stata scelta una variante dell'ottimizzatore Adam, nota come AdamW. A differenza di Adam classico, AdamW disaccoppia il meccanismo di weight decay dalla gradiente, portando spesso a una migliore generalizzazione e performance più stabili. L'opzione `amsgrad` è attivata per migliorare ulteriormente la convergenza.
-   **Istanza della Loss**: Viene creata un'istanza della nostra funzione di loss personalizzata, `ContrastativeFocalLoss`, con i parametri `alpha`, `gamma` e `contrast_weight` specificati.

#### Scheduler del Learning Rate: `CosineAnnealingWarmRestarts`
Per evitare di rimanere bloccati in minimi locali e per esplorare meglio lo spazio della loss, viene utilizzato uno scheduler avanzato. Il `CosineAnnealingWarmRestarts` varia ciclicamente il learning rate (LR) seguendo un andamento cosinusoidale:
-   **Decadimento Coseno**: L'LR diminuisce gradualmente da un valore massimo a un valore minimo (`eta_min`) in un certo numero di epoche (`T_0`).
-   **Warm Restarts**: Al termine di ogni ciclo, l'LR viene "resettato" al suo valore iniziale. Questo "calcio" aiuta il modello a uscire da eventuali plateau e a convergere verso soluzioni migliori.
-   **Cicli Crescenti**: Il parametro `T_mult=2` fa sì che la durata di ogni ciclo successivo raddoppi, permettendo al modello di affinare la sua convergenza man mano che l'addestramento procede.

#### Classe `EarlyStopping`
Per evitare di addestrare il modello più del necessario (rischiando overfitting e spreco di tempo), viene implementata una classe `EarlyStopping`. Questa utility monitora una metrica di performance sul set di validazione (nel nostro caso, l'F1-score) e interrompe l'addestramento se non si osservano miglioramenti per un numero specificato di epoche (`patience`).

Le sue responsabilità principali sono:
-   **Monitorare lo Score**: Confronta lo score dell'epoca corrente con il miglior score ottenuto finora.
-   **Salvare il Modello Migliore**: Se viene raggiunto un nuovo score migliore, salva un checkpoint del modello, dello stato dell'ottimizzatore e delle metriche correnti.
-   **Interrompere l'Addestramento**: Se non ci sono miglioramenti per `patience` epoche consecutive, imposta un flag per terminare il ciclo di training.

In [13]:
class EarlyStopping:
    def __init__(self, patience=15, min_delta=0, mode='max', save_path='best_model.pth'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.save_path = save_path
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, score, model, optimizer, epoch, **kwargs):  #  Aggiunto **kwargs
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model, optimizer, epoch, score, **kwargs)
            return False

        # Fix logica improved
        if self.mode == "max":
            improved = score > (self.best_score + self.min_delta)
        else:  # mode == "min"
            improved = score < (self.best_score - self.min_delta)  #  Corretto

        if improved:
            self.best_score = score
            self.counter = 0
            self.save_checkpoint(model, optimizer, epoch, score, **kwargs)
            return False
        else:
            self.counter += 1  # Aggiunto self.
            print(f"Nessun miglioramento (counter {self.counter}/{self.patience})")
            if self.counter >= self.patience:
                self.early_stop = True
                return True
            return False

    def save_checkpoint(self, model, optimizer, epoch, score, **kwargs):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'score': score,
            **kwargs
        }
        torch.save(checkpoint, self.save_path)
        print(f"Checkpoint salvato! Score: {score:.4f}")

In [14]:
optimizer = optim.AdamW(
  model.parameters(),
  lr,
  betas=(0.9, 0.999),
  weight_decay=weight_decay,
  amsgrad=True
)

scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,  # Restart ogni 10 epochs
    T_mult=2,
    eta_min=1e-6
)

criterion = ContrastativeFocalLoss(
    alpha=0.25,
    gamma=2.0,
    contrast_weight=0.3
)

### Ciclo di Addestramento e Valutazione

Questa sezione finale contiene il cuore pulsante del progetto: le funzioni che definiscono il ciclo di addestramento e valutazione del modello. Il codice è strutturato in tre parti principali per garantire chiarezza e modularità.

#### Funzione di Addestramento per Epoca (`train_one_epoch`)
Questa funzione incapsula tutta la logica necessaria per eseguire una singola epoca di addestramento. Le sue responsabilità includono:
-   **Iterazione sui Dati**: Scorre attraverso i batch forniti dal `train_loader`.
-   **Passaggio Forward e Calcolo della Loss**: Esegue il passaggio forward del modello per ottenere sia i *logit* sia le *feature*, che vengono poi usati per calcolare la nostra `ContrastativeFocalLoss` ibrida.
-   **Mixed Precision Training**: Per accelerare l'addestramento e ridurre l'uso di memoria della GPU, viene utilizzata la precisione mista (`torch.autocast`). Il `GradScaler` gestisce in modo sicuro il calcolo dei gradienti per prevenire problemi di underflow numerico.
-   **Backpropagation e Ottimizzazione**: Calcola i gradienti, applica il **Gradient Clipping** (con `max_norm=1.0`) per prevenire gradienti esplosivi e stabilizzare l'addestramento, e infine aggiorna i pesi del modello.
-   **Calcolo delle Metriche**: Restituisce la loss media e l'accuracy per l'epoca di addestramento.

#### Funzione di Valutazione (`testing`)
Questa funzione è dedicata a valutare le performance del modello sul set di test (o validazione) al termine di ogni epoca. È decorata con `@torch.no_grad()` per disattivare il calcolo dei gradienti, rendendo l'inferenza più veloce e sicura.
-   **Modalità di Valutazione**: Imposta il modello in `model.eval()` per disattivare strati come Dropout e BatchNorm.
-   **Calcolo di Metriche Complete**: Oltre alla loss e all'accuracy, calcola un set completo di metriche di classificazione utilizzando `scikit-learn`:
    -   **Precision, Recall, F1-Score**: Per una valutazione approfondita dell'equilibrio del classificatore.
    -   **AUC (Area Under the Curve)**: Per misurare la capacità del modello di distinguere tra le classi.
    -   **Matrice di Confusione**: Per visualizzare in dettaglio gli errori di classificazione (veri positivi, falsi negativi, ecc.).
-   **Output Strutturato**: Restituisce un dizionario contenente tutte le metriche calcolate e l'oggetto della matrice di confusione.

#### Ciclo Principale di Addestramento (`train_model`)
Questa è la funzione orchestratrice che esegue l'intero processo di addestramento per un numero specificato di epoche.
-   **Inizializzazione**: Prepara il `GradScaler` e l'istanza della classe `EarlyStopping`.
-   **Loop sulle Epoche**: Per ogni epoca, esegue in sequenza `train_one_epoch` e `testing`.
-   **Aggiornamento dello Scheduler**: Dopo ogni epoca di valutazione, aggiorna il learning rate secondo la strategia del `CosineAnnealingWarmRestarts`.
-   **Logging e Monitoraggio**: Stampa un riepilogo dettagliato delle performance di training e test, inclusi F1-score, AUC e il learning rate corrente. Mostra anche la matrice di confusione.
-   **Controllo Early Stopping**: Utilizza l'**F1-score** sul set di test come metrica chiave per decidere se l'addestramento deve continuare. Se non ci sono miglioramenti per un numero di epoche pari a `patience`, il ciclo viene interrotto e il modello con le migliori performance viene conservato.

In [15]:
def train_one_epoch(model, loader, criterion, optimizer, scaler, device, use_mixed_precision):
  model.train()

  total_loss = 0.0
  correct,total = 0,0

  for rgb,depth,label in tqdm(loader,desc="Training",leave=False):
    rgb, depth, label = rgb.to(device), depth.to(device), label.to(device)
    optimizer.zero_grad()

    if use_mixed_precision and scaler is not None:
        with torch.autocast(device_type='cuda'):
            out, features = model(rgb, depth, return_features=True)
            loss = criterion(out, features, label)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        #FIX: Calcola accuracy FUORI dal context autocast
        with torch.no_grad():
            # Ricalcola forward in float32 per accuracy stabile
            out_eval = model(rgb, depth, return_features=False)
            preds = out_eval.argmax(1)
            correct += (preds == label).sum().item()
    else:
        out, features = model(rgb, depth, return_features=True)
        loss = criterion(out, features, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Accuracy calculation
        preds = out.argmax(1)
        correct += (preds == label).sum().item()

    total_loss += loss.item()
    total += label.size(0)

  avg_loss = total_loss / len(loader)
  accuracy = 100 * correct / total
  return avg_loss, accuracy

@torch.no_grad()
def testing(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct, total = 0, 0
    all_preds, all_labels, all_probs = [], [], []

    for rgb, depth, label in tqdm(loader, desc="Evaluating", leave=False):
        rgb, depth, label = rgb.to(device), depth.to(device), label.to(device)

        out = model(rgb, depth, return_features=False)
        loss = criterion(out, label)
        total_loss += loss.item()

        probs = F.softmax(out, dim=1)
        preds = out.argmax(1)

        correct += (preds == label).sum().item()
        total += label.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(label.cpu().numpy())
        #FIX: Prendi SOLO la probabilità della classe 1 (fake)
        all_probs.extend(probs[:, 1].cpu().numpy())  # Non probs.cpu().numpy()

    metrics = {
        'loss': total_loss / len(loader),
        'accuracy': 100 * correct / total,
        'precision': precision_score(all_labels, all_preds, zero_division=0),
        'recall': recall_score(all_labels, all_preds, zero_division=0),
        'f1': f1_score(all_labels, all_preds, zero_division=0),
        'auc': roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else 0.0,
        'predictions': all_preds,
        'labels': all_labels
    }

    cm = confusion_matrix(all_labels, all_preds)

    return metrics, cm


def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler,
                device, epochs=100, patience=15, use_mixed_precision=True, save_path='best_model.pth'):

  scaler = GradScaler() if use_mixed_precision else None
  best_loss = float('inf')
  early_stopping = EarlyStopping()

  for epoch in range(epochs):
      print(f"\nEpoch {epoch+1}/{epochs}")
      print("-" * 40)

      train_loss,train_acc = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device, use_mixed_precision)
      test_metrics, cm = testing(model, test_loader, criterion, device)

      scheduler.step()

      # Stampa metriche
      print(f"   Train → Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%")
      print(f"   Test  → Loss: {test_metrics['loss']:.4f} | Acc: {test_metrics['accuracy']:.2f}%")
      print(f"   Metrics → P: {test_metrics['precision']:.3f} | R: {test_metrics['recall']:.3f} | F1: {test_metrics['f1']:.3f} | AUC: {test_metrics['auc']:.3f}")
      print(f"   LR: {optimizer.param_groups[0]['lr']:.6f}")
      print(f"\nConfusion matrix: \n{cm}\n")

      # Early stopping (usa F1 come metrica principale)
      should_stop = early_stopping(
          test_metrics['f1'],
          model,
          optimizer,
          epoch,
          test_acc=test_metrics['accuracy'],
          f1=test_metrics['f1']
      )

      if not should_stop:
          print(f"No improvement: {early_stopping.counter}/{patience}")

      if should_stop:
          print(f"\nEarly stopping triggered at epoch {epoch+1}")
          break

  print(f"\n{'='*60}")
  print(f"Training completato!")
  print(f"{'='*60}\n")

  return early_stopping.best_score


In [16]:
best_f1 = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=100,
    patience=15,
    use_mixed_precision=use_mixed_precision,
    save_path="best_antispoofing_improved.pth"
)


Epoch 1/100
----------------------------------------


   Train → Loss: 1.0560 | Acc: 51.50%
   Test  → Loss: 0.0450 | Acc: 47.43%
   Metrics → P: 0.964 | R: 0.317 | F1: 0.477 | AUC: 0.813
   LR: 0.000098

Confusion matrix: 
[[389  15]
 [855 396]]

Checkpoint salvato! Score: 0.4765
No improvement: 0/15

Epoch 2/100
----------------------------------------


   Train → Loss: 1.0504 | Acc: 55.11%
   Test  → Loss: 0.0394 | Acc: 62.05%
   Metrics → P: 0.998 | R: 0.499 | F1: 0.665 | AUC: 0.936
   LR: 0.000091

Confusion matrix: 
[[403   1]
 [627 624]]

Checkpoint salvato! Score: 0.6652
No improvement: 0/15

Epoch 3/100
----------------------------------------


   Train → Loss: 1.0449 | Acc: 57.68%
   Test  → Loss: 0.0359 | Acc: 68.34%
   Metrics → P: 1.000 | R: 0.581 | F1: 0.735 | AUC: 0.953
   LR: 0.000080

Confusion matrix: 
[[404   0]
 [524 727]]

Checkpoint salvato! Score: 0.7351
No improvement: 0/15

Epoch 4/100
----------------------------------------


   Train → Loss: 1.0390 | Acc: 63.83%
   Test  → Loss: 0.0350 | Acc: 73.35%
   Metrics → P: 0.999 | R: 0.648 | F1: 0.786 | AUC: 0.961
   LR: 0.000066

Confusion matrix: 
[[403   1]
 [440 811]]

Checkpoint salvato! Score: 0.7862
No improvement: 0/15

Epoch 5/100
----------------------------------------


...

   Train → Loss: 0.8614 | Acc: 97.38%
   Test  → Loss: 0.0004 | Acc: 99.82%
   Metrics → P: 0.999 | R: 0.998 | F1: 0.999 | AUC: 1.000
   LR: 0.000032

Confusion matrix: 
[[ 403    1]
 [   2 1249]]

Nessun miglioramento (counter 11/15)
No improvement: 11/15

Epoch 56/100
----------------------------------------


   Train → Loss: 0.8585 | Acc: 97.38%
   Test  → Loss: 0.0003 | Acc: 99.94%
   Metrics → P: 0.999 | R: 1.000 | F1: 1.000 | AUC: 1.000
   LR: 0.000028

Confusion matrix: 
[[ 403    1]
 [   0 1251]]

Nessun miglioramento (counter 12/15)
No improvement: 12/15

Epoch 57/100
----------------------------------------


   Train → Loss: 0.8567 | Acc: 97.34%
   Test  → Loss: 0.0003 | Acc: 99.94%
   Metrics → P: 0.999 | R: 1.000 | F1: 1.000 | AUC: 1.000
   LR: 0.000025

Confusion matrix: 
[[ 403    1]
 [   0 1251]]

Nessun miglioramento (counter 13/15)
No improvement: 13/15

Epoch 58/100
----------------------------------------


   Train → Loss: 0.8633 | Acc: 97.18%
   Test  → Loss: 0.0003 | Acc: 99.94%
   Metrics → P: 0.999 | R: 1.000 | F1: 1.000 | AUC: 1.000
   LR: 0.000021

Confusion matrix: 
[[ 403    1]
 [   0 1251]]

Nessun miglioramento (counter 14/15)
No improvement: 14/15

Epoch 59/100
----------------------------------------


   Train → Loss: 0.8568 | Acc: 97.72%
   Test  → Loss: 0.0013 | Acc: 99.58%
   Metrics → P: 0.999 | R: 0.995 | F1: 0.997 | AUC: 1.000
   LR: 0.000018

Confusion matrix: 
[[ 403    1]
 [   6 1245]]

Nessun miglioramento (counter 15/15)

Early stopping triggered at epoch 59

Training completato!

